In [4]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
# import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
import math
import time
# from sympy import *
from scipy.linalg import schur, eigvals
extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

# Listing all statistical properties
* ## rank-2 eigen spectrum, SVD
* ## rank-1 left $\mathbf{n}$ and right $\mathbf{m}$

In [63]:
''' heterogeneous case with single zero g_{EE} '''
# strfile = '2021-01-19-11_23_52_heterogeneousNotgEE_cloudsdata'
strfile ='2021-01-19-12_02_53_heterogeneousgII_cloudsdata'
# strfile='2021-01-19-11_44_14_homogeneous_cloudsdata'
data = np.load(strfile+'.npz')#'2021-01-19-12_02_53_heterogeneousgII_cloudsdata.npz')
meigseries,neigseries=data['meigseries'],data['neigseries']
gaverageseries=data['gaverageseries']
# # loading calculated mean and std (2*2) for rank1, 0,0 m,E 0,1 m,I
# meanloadingseries,stdloadingseries=data['meanloadingseries'],data['stdloadingseries']
# ((niter,ntrials,NE*2))
ngavg,nbatch,_=np.shape(meigseries)

# Nt=np.array([1000,1000])
# NE,NI=Nt[0],Nt[1]
# Nparams=np.array([NE,NI])
# JE,JI,a,b=1.0,3.5,-0.6,0.6
# a,b=0.0,0.0
# JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
# Jparameters = np.zeros(4+1)
# Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
# Jparameters[4]=NE
nrank=1
Jparameters=data['Jparameters']
NE = int(Jparameters[4])

P = np.zeros((2,2))
P = np.array([[Jparameters[0],-Jparameters[1]],[Jparameters[2],-Jparameters[3]]])
Am=np.zeros((NE*2,NE*2))
Am[:NE,:NE],Am[:NE,NE:]=P[0,0]/NE,P[0,1]/NE
Am[NE:,:NE],Am[NE:,NE:]=P[1,0]/NE,P[1,1]/NE
# # calculate the original \bar{m} and \bar{n}
# uAm,svAm,vhAm=la.svd(Am)
# vAm=vhAm.T
# mAm,nAm=uAm[:,:nrank],vAm[:,:nrank]
# for i in range(nrank):
#     nAm[:,i]*=svAm[i]
# print('\bar{m}&\bar{n}')
# print(mAm[1::NE,0],nAm[1::NE,0])
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
print('\bar{m}&\bar{n}')
print(meigvecAm[1::NE,0],neigvecAm[1::NE,0])
print('eigenvalues:',eigvAm[:nrank])
# [-0.02236068 -0.02236068] [-0.04472136  0.15652476] #strfile='2021-01-19-11_44_14_homogeneous_cloudsdata'
# print(np.sum(neigvecAm[:,0]*meigvecAm[:,0]))


ar{m}ar{n}
[0.02236068+0.j 0.02236068+0.j] [ 0.04472136-0.j -0.15652476+0.j]
eigenvalues: [-2.5+0.j]


In [253]:

''' RANK-2 '''
''' heterogeneous case with single zero g_{EE} '''
# strfile = '2021-01-20-10_38_16_heterogeneousgIIR2_cloudsdata'
# strfile = '2021-01-20-11_12_48_homogeneousR2_cloudsdata'
strfile ='2021-01-20-11_41_35_heterogeneousNotgEER2_cloudsdata'
# strfile='2021-01-23-16_18_40_heterogeneousNotgEER2_cloudsdata'
# strfile = '2021-01-23-17_02_52_homogeneousR2_cloudsdata'
data = np.load(strfile+'.npz')#'2021-01-19-12_02_53_heterogeneousgII_cloudsdata.npz')
meigseries,neigseries=data['meigseries'],data['neigseries']
gaverageseries=data['gaverageseries']
# # loading calculated mean and std (2*2) for rank1, 0,0 m,E 0,1 m,I
# meanloadingseries,stdloadingseries=data['meanloadingseries'],data['stdloadingseries']
# ((niter,ntrials,NE*2))
ngavg,nbatch,_,_=np.shape(meigseries)
nrank=2
Jparameters=data['Jparameters']
NE = int(Jparameters[4])

P = np.zeros((2,2))
P = np.array([[Jparameters[0],-Jparameters[1]],[Jparameters[2],-Jparameters[3]]])
Am=np.zeros((NE*2,NE*2))
Am[:NE,:NE],Am[:NE,NE:]=P[0,0]/NE,P[0,1]/NE
Am[NE:,:NE],Am[NE:,NE:]=P[1,0]/NE,P[1,1]/NE
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
print('\bar{m}&\bar{n}')
print(meigvecAm[1::NE,0],meigvecAm[1::NE,1])
print(neigvecAm[1::NE,0],neigvecAm[1::NE,1])
print('eigenvalues:',eigvAm[:nrank])
print(np.sum(neigvecAm[:,0]*meigvecAm[:,0]))
print(np.sum(neigvecAm[:,1]*meigvecAm[:,1]))

ar{m}ar{n}
[-0.01826104+0.j -0.02581733+0.j] [-0.02236068+0.j -0.02236068+0.j]
[-0.42348816+0.j  0.42348816-0.j] [ 0.30559596-0.j -0.21615324+0.j]
eigenvalues: [-3.2+0.j -2. +0.j]
(-3.200000000000064+0j)
(-1.999999999999904+0j)


In [254]:
''' For rank 2 '''
# calculate \theta overlap
loadingeigvec=np.zeros((ngavg,2,nrank*2,NE,nbatch)) # ng,EI,nrank*2 mn,Npopulation,ntrials
loadingeigvecAm=np.zeros((2,nrank*2,NE))
meanloadingeigclouds,stdloadingeigclouds = np.zeros((ngavg,2,nrank*2,nbatch)),np.zeros((ngavg,2,nrank*2,nbatch))
Norm2=np.zeros((ngavg,3,nrank*2,nbatch))
Norm2Am=np.zeros((3,nrank*2,nbatch))
overlapeig,overlapeig_renorm=np.zeros((ngavg,3,nrank*2,nbatch)),np.zeros((ngavg,3,nrank*2,nbatch))
# lambda reconstruction
lambda_reconstruct=np.zeros((ngavg,3,nrank,nbatch)) # E pop, I pop, overall
for j in range(nrank):
    # eig
    loadingeigvecAm[0,j,:]=np.squeeze(meigvecAm[:NE,j]).T # m_j^E
    loadingeigvecAm[0,j+nrank,:]=np.squeeze(neigvecAm[:NE,j]).T # n_j^E
    loadingeigvecAm[1,j,:]=np.squeeze(meigvecAm[NE:,j]).T # m_j^I
    loadingeigvecAm[1,j+nrank,:]=np.squeeze(neigvecAm[NE:,j]).T # n_j^I
    Norm2Am[0,j]=np.sum(np.squeeze(loadingeigvecAm[0,j,:])**2,axis=0) # excitatory
    Norm2Am[1,j]=np.sum(np.squeeze(loadingeigvecAm[1,j,:])**2,axis=0) # inhibitory
    Norm2Am[2,j]=Norm2Am[1,j]+Norm2Am[0,j] # overall
    Norm2Am[0,j+nrank]=np.sum(np.squeeze(loadingeigvecAm[0,j+nrank,:])**2,axis=0) # excitatory
    Norm2Am[1,j+nrank]=np.sum(np.squeeze(loadingeigvecAm[1,j+nrank,:])**2,axis=0) # inhibitory
    Norm2Am[2,j+nrank]=Norm2Am[1,j+nrank]+Norm2Am[0,j+nrank] # overall
for j in range(nrank):
    for i in range(ngavg):
        # eig
        loadingeigvec[i,0,j,:,:]=np.squeeze(meigseries[i,:,:NE,j]).T # m_j^E
        loadingeigvec[i,0,j+nrank,:,:]=np.squeeze(neigseries[i,:,:NE,j]).T # n_j^E
        loadingeigvec[i,1,j,:,:]=np.squeeze(meigseries[i,:,NE:,j]).T # m_j^I
        loadingeigvec[i,1,j+nrank,:,:]=np.squeeze(neigseries[i,:,NE:,j]).T # n_j^I

        # renormalization
        # m, right eigenvectors
        Norm2[i,0,j,:]=np.sum(np.squeeze(loadingeigvec[i,0,j,:,:])**2,axis=0) # excitatory
        Norm2[i,1,j,:]=np.sum(np.squeeze(loadingeigvec[i,1,j,:,:])**2,axis=0) # inhibitory
        Norm2[i,2,j,:]=Norm2[i,1,j,:]+Norm2[i,0,j,:] # overall
        # n, left eigenvectors
        Norm2[i,0,j+nrank,:]=np.sum(np.squeeze(loadingeigvec[i,0,j+nrank,:,:])**2,axis=0) # excitatory n
        Norm2[i,1,j+nrank,:]=np.sum(np.squeeze(loadingeigvec[i,1,j+nrank,:,:])**2,axis=0) # inhibitory n
        Norm2[i,2,j+nrank,:]=Norm2[i,1,j+nrank,:]+Norm2[i,0,j+nrank,:] # overall normalization
        # overlap
        # m
        overlapeig[i,0,j,:]=np.squeeze(np.reshape(meigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j,:,:])) # excitatory overlap -- renormalized
        overlapeig[i,1,j,:]=np.squeeze(np.reshape(meigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j,:,:])) # inhibitory overlap -- renromalized
        overlapeig[i,2,j,:]=np.squeeze(np.reshape(meigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j,:,:]))+np.squeeze(np.reshape(meigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j,:,:])) # inhibitory overlap -- renormalized
        # n
        overlapeig[i,0,j+nrank,:]=np.squeeze(np.reshape(neigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j+nrank,:,:]))/(eigvAm[j])**2 # excitatory overlap -- renormalized
        overlapeig[i,1,j+nrank,:]=np.squeeze(np.reshape(neigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j+nrank,:,:]))/(eigvAm[j])**2# inhibitory overlap -- renormalize+
        overlapeig[i,2,j+nrank,:]=np.squeeze(np.reshape(neigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j+nrank,:,:]))/(eigvAm[j])**2+np.squeeze(np.reshape(neigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j+nrank,:,:]))/(eigvAm[j])**2# inhibitory overlap -- renormalized

        # overlap renormalizing
        # m
        overlapeig_renorm[i,0,j,:]=np.squeeze(np.reshape(meigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j,:,:]))/np.sqrt(Norm2[i,0,j,:])/np.sqrt(Norm2Am[0,j]) # excitatory overlap -- renormalized
        overlapeig_renorm[i,1,j,:]=np.squeeze(np.reshape(meigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j,:,:]))/np.sqrt(Norm2[i,1,j,:])/np.sqrt(Norm2Am[1,j]) # inhibitory overlap -- renromalized
        overlapeig_renorm[i,2,j,:]=(np.squeeze(np.reshape(meigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j,:,:]))+np.squeeze(np.reshape(meigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j,:,:])))/np.sqrt(Norm2[i,2,j,:])/np.sqrt(Norm2Am[2,j]) # overlap -- renromalized
        # n
        overlapeig_renorm[i,0,j+nrank,:]=np.squeeze(np.reshape(neigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j+nrank,:,:]))/np.sqrt(Norm2[i,0,j+nrank,:])/np.sqrt(Norm2Am[0,j+nrank]) # excitatory overlap -- renormalized
        overlapeig_renorm[i,1,j+nrank,:]=np.squeeze(np.reshape(neigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j+nrank,:,:]))/np.sqrt(Norm2[i,1,j+nrank,:])/np.sqrt(Norm2Am[1,j+nrank]) # inhibitory overlap -- renormalized
        overlapeig_renorm[i,2,j+nrank,:]=(np.squeeze(np.reshape(neigvecAm[:NE,j],(1,NE))@np.squeeze(loadingeigvec[i,0,j+nrank,:,:]))+np.squeeze(np.reshape(neigvecAm[NE:,j],(1,NE))@np.squeeze(loadingeigvec[i,1,j+nrank,:,:])))/np.sqrt(Norm2[i,2,j+nrank,:])/np.sqrt(Norm2Am[2,j+nrank])
        # statistics
        meanloadingeigclouds[i,0,j,:]=(np.mean(np.squeeze(loadingeigvec[i,0,j,:,:]),axis=0)) # m excitatory trial averaging
        meanloadingeigclouds[i,0,j+nrank,:]=(np.mean(np.squeeze(loadingeigvec[i,0,j+nrank,:,:]),axis=0)) # n excitatory trial average
        meanloadingeigclouds[i,1,j,:]=(np.mean(np.squeeze(loadingeigvec[i,1,j,:,:]),axis=0)) # m inhibitory trial average
        meanloadingeigclouds[i,1,j+nrank,:]=(np.mean(np.squeeze(loadingeigvec[i,1,j+nrank,:,:]),axis=0)) # n inhibitory trial average

        stdloadingeigclouds[i,0,j,:]=np.std(np.squeeze(loadingeigvec[i,0,j,:,:]),axis=0)
        stdloadingeigclouds[i,0,j+nrank,:]=np.std(np.squeeze(loadingeigvec[i,0,j+nrank,:,:]),axis=0)
        stdloadingeigclouds[i,1,j,:]=np.std(np.squeeze(loadingeigvec[i,1,j,:,:]),axis=0)
        stdloadingeigclouds[i,1,j+nrank,:]=np.std(np.squeeze(loadingeigvec[i,1,j+nrank,:,:]),axis=0)

        # reproduce lambda overall
        lambda_reconstruct[i,0,j,:]=np.sum(np.squeeze(meigseries[i,:,:NE,j])*np.squeeze(neigseries[i,:,:NE,j]),axis=1)
        lambda_reconstruct[i,1,j,:]=np.sum(np.squeeze(meigseries[i,:,NE:,j])*np.squeeze(neigseries[i,:,NE:,j]),axis=1)
        lambda_reconstruct[i,2,j,:]=np.sum(np.squeeze(meigseries[i,:,:,j])*np.squeeze(neigseries[i,:,:,j]),axis=1)

## Plottings for eigenvectors 
* overlap of right and left eigenvectors -- $\lambda_1,\lambda_2$
* overlap of the new and original eigenvectors -- renormalize?  
* connectivity patterns in loading space -- mean and variance of gaussian pattern

In [255]:
yticks = np.linspace(-1.5,1.50,3)
ylims = [-1.5,1.5]
fig,ax=plt.subplots(2,3,figsize=(12,4))
# FIGURE 1
for i in range(ngavg):
    for j in range(nrank):
        # RECONSTRUCT EIGENVALUES
         # m1
        ax[j][0].scatter(gaverageseries[i]*np.ones(nbatch),lambda_reconstruct[i,0,j,:],s=5,color='red',alpha=0.5)
        ax[j][0].scatter(gaverageseries[i]*np.ones(nbatch),lambda_reconstruct[i,1,j,:],s=5,color='blue',alpha=0.5)
        ax[j][0].scatter(gaverageseries[i]*np.ones(nbatch),lambda_reconstruct[i,2,j,:],s=5,color='black',alpha=0.5)
        overlapeig_renorm[i,0,j,:]
                # OVERLAP
        # m1
        # ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),overlapeig[i,0,j,:],s=5,color='red',alpha=0.5)
        # ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),overlapeig[i,1,j,:],s=5,color='blue',alpha=0.5)

        ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),overlapeig_renorm[i,0,j,:],s=5,color='red',alpha=0.5)
        ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),overlapeig_renorm[i,1,j,:],s=5,color='blue',alpha=0.5)
        ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),overlapeig_renorm[i,2,j,:],s=5,color='black',alpha=0.5)

        # n1
        ax[j][2].scatter(gaverageseries[i]*np.ones(nbatch),overlapeig_renorm[i,0,j+nrank,:],s=5,color='red',alpha=0.5)
        ax[j][2].scatter(gaverageseries[i]*np.ones(nbatch),overlapeig_renorm[i,1,j+nrank,:],s=5,color='blue',alpha=0.5)
        ax[j][2].scatter(gaverageseries[i]*np.ones(nbatch),overlapeig_renorm[i,2,j+nrank,:],s=5,color='black',alpha=0.5)


# ax[0][0].set_ylim(ylims)
# ax[1][0].set_ylim(ylims)

In [17]:
print(np.mean(lambda_reconstruct[:,2,0,:]))

[-3.19848481 -3.19560433 -3.23030985 -3.21675155 -3.21975556 -3.17560632
 -3.2131477  -3.0465627  -3.17194874 -3.23273991 -3.14884097 -3.18938156
 -3.20051234 -3.21066659 -3.19976068 -3.18586634 -3.20226587 -3.19118601
 -3.2157759  -3.16754911]


In [256]:
yticks = np.linspace(-1.5,1.50,3)
ylims = [-1.5,1.5]
fig,ax=plt.subplots(2,2,figsize=(12,4))
# FIGURE same as overlap
for i in range(ngavg):
    for j in range(nrank):
        # mean m1
        ax[j][0].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,0,j,:])/np.sqrt(Norm2[i,0,j,:]),s=5,color='red',alpha=0.5)
        ax[j][0].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,1,j,:])/np.sqrt(Norm2[i,1,j,:]),s=5,color='blue',alpha=0.5)
        # mean n1
        ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,0,nrank+j,:])/np.sqrt(Norm2[i,0,j+nrank,:]),s=5,color='red',alpha=0.5)
        ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,1,nrank+j,:])/np.sqrt(Norm2[i,1,j+nrank,:]),s=5,color='blue',alpha=0.5)

fig,ax=plt.subplots(2,2,figsize=(12,4))
# FIGURE absolute value
for i in range(ngavg):
    for j in range(nrank):
        # mean m1
        ax[j][0].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,0,j,:]),s=5,color='red',alpha=0.5)
        ax[j][0].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,1,j,:]),s=5,color='blue',alpha=0.5)
        # mean n1
        ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,0,nrank+j,:]),s=5,color='red',alpha=0.5)
        ax[j][1].scatter(gaverageseries[i]*np.ones(nbatch),np.abs(meanloadingeigclouds[i,1,nrank+j,:]),s=5,color='blue',alpha=0.5)

## quantitative measurements

In [257]:
# statistics
# meanloadingeigclouds[i,0,j,:]=(np.mean(np.squeeze(loadingeigvec[i,0,j,:,:]),axis=0)) # m excitatory trial averaging
# meanloadingeigclouds[i,0,j+nrank,:]=(np.mean(np.squeeze(loadingeigvec[i,0,j+nrank,:,:]),axis=0)) # n excitatory trial average
# meanloadingeigclouds[i,1,j,:]=(np.mean(np.squeeze(loadingeigvec[i,1,j,:,:]),axis=0)) # m inhibitory trial average
# meanloadingeigclouds[i,1,j+nrank,:]=(np.mean(np.squeeze(loadingeigvec[i,1,j+nrank,:,:]),axis=0)) # n inhibitory trial average
mAm2,nAm2=np.zeros((2,2)),np.zeros((2,2))
for i in range(nrank):
    mAm2[:,i]=meigvecAm[1::NE,i]
    nAm2[:,i]=neigvecAm[1::NE,i]
invmbar,invnbar=la.inv(mAm2),la.inv(nAm2)
# m1--
meanloadingeig_trialsavg[:,:,0] = -np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,0,:])),axis=2)#np.squeeze(meanloadingeigclouds[:,:,:,12])#
meanloadingeig_trialsavg[:,:,1] = np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,1,:])),axis=2)
meanloadingeig_trialsavg[:,:,2] = np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,2,:])),axis=2)
meanloadingeig_trialsavg[:,0,2]*=-1
meanloadingeig_trialsavg[:,:,3] = np.mean(np.abs(np.squeeze(meanloadingeigclouds[:,:,3,:])),axis=2)
meanloadingeig_trialsavg[:,1,3]*=-1
Oeigseries=np.zeros((ngavg,2,nrank*2))
for i in range(ngavg):
    m_mean = np.squeeze(meanloadingeig_trialsavg[i,:,:nrank])
    n_mean = np.squeeze(meanloadingeig_trialsavg[i,:,nrank:])
    Oeigm,Oeign = invmbar@m_mean,invnbar@n_mean
    Oeigseries[i,:,:nrank],Oeigseries[i,:,nrank:]=Oeigm.copy(),Oeign.copy()




In [258]:
for i in range(20):
    mEnum, mInum=Oeigseries[i,:,:nrank],Oeigseries[i,:,nrank:]
    print(mEnum@mInum.T)

[[ 1.00075099  0.00494783]
 [-0.00269889 -0.99904399]]
[[ 1.00089434  0.00422734]
 [-0.00319749 -0.99925597]]
[[ 1.00152747  0.00399406]
 [-0.00617696 -0.99913241]]
[[ 1.00096882  0.00274053]
 [ 0.00266788 -1.00113915]]
[[ 1.00218527 -0.00211217]
 [ 0.00571484 -1.00443599]]
[[ 1.00785861  0.02640358]
 [-0.02243032 -0.99563013]]
[[ 1.00136277e+00  1.14155006e-02]
 [ 5.28067296e-04 -1.00058445e+00]]
[[ 1.00612545 -0.00785679]
 [ 0.01047373 -1.00984077]]
[[ 1.01117012  0.03289877]
 [-0.02729409 -0.99485047]]
[[ 1.00610158 -0.03966515]
 [ 0.02212081 -1.01940298]]
[[ 1.00056596 -0.04876943]
 [ 0.02311919 -1.01489564]]
[[ 1.00491048 -0.0470396 ]
 [ 0.03738602 -1.02232827]]
[[ 1.01379089 -0.02432074]
 [ 0.02120981 -1.02240279]]
[[ 1.08895068 -0.27138443]
 [ 0.17863451 -1.19129435]]
[[ 1.00516988 -0.05076952]
 [ 0.04357295 -1.03005291]]
[[ 1.03782274 -0.0974096 ]
 [ 0.05483822 -1.06784983]]
[[ 1.00610598 -0.16531026]
 [ 0.10080351 -1.06732237]]
[[ 1.01760963e+00  4.07695444e-04]
 [-1.33954787e

In [259]:
# three copy
idx = 12
grand = gaverageseries[idx]*np.ones(4)*np.sqrt(2.0/3.0)
grand[0]=0
xee,xei,xie,xii=grand[0],grand[1],grand[2],grand[3]
gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
eigvgm,eigvecgm=la.eig(gmat)
# print(eigvgm)
print(Oeigseries[idx,:,:nrank])
print(Oeigseries[idx,:,nrank:])
attt=(1- gaverageseries[idx]**2/eigvAm[:nrank]**2)#eigvgm**2/eigvAm[:nrank]**2)
print(np.sqrt(attt))

[[ 0.97225956 -0.00136439]
 [ 0.00438349 -0.94326039]]
[[ 1.04269155 -0.01764007]
 [-0.02349375  1.08379384]]
[0.97915281+0.j 0.94571402+0.j]


In [261]:
## scatter and direction
nE1,nI1=np.squeeze(loadingeigvec[10,0,2,:NE,10]),np.squeeze(loadingeigvec[10,1,2,:,10])
nE2,nI2=np.squeeze(loadingeigvec[10,0,3,:NE,10]),np.squeeze(loadingeigvec[10,1,3,:,10])
# fig,ax=plt.subplots(figsize=(4,4))
# FIGURE same as overlap
ax.scatter(nE1,nE2,s=5,color='red',alpha=0.5)
ax.scatter(nI1,nI2,s=5,color='blue',alpha=0.5)
# for i in range(2):
#     nAm2[:,i]=nAm2[:,i]/eigvAm[i]
covnAm=nAm2.T@nAm2
_,eigveccov=la.eig(covnAm)
ax.plot([0,eigveccov[0,0]],[0,eigveccov[1,0]],'pink')